# Spooky Author Identification (NLP)
https://www.kaggle.com/competitions/spooky-author-identification

The main goal is to use machine learning to predict the author of excerpts from horror stories by Edgar Allan Poe, Mary Shelley, and HP Lovecraft.

In [2]:
import nltk
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

## Getting Started
https://www.kaggle.com/code/rtatman/beginner-s-tutorial-python